In [1]:
import numpy as np
import pandas as pd
# import requests
# from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer , sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
lem = WordNetLemmatizer()

In [2]:
input_df = pd.read_excel('../Input.xlsx')

In [3]:
input_df.head(5)

,URL_ID,URL
0,1,https://insights.blackcoffer.com/is-telehealth...
1,2,https://insights.blackcoffer.com/how-telehealt...
2,3,https://insights.blackcoffer.com/is-telemedici...
3,4,https://insights.blackcoffer.com/is-telehealth...
4,5,https://insights.blackcoffer.com/how-people-di...


In [4]:
URLs = input_df['URL']

In [5]:
URLs

0      https://insights.blackcoffer.com/is-telehealth...
1      https://insights.blackcoffer.com/how-telehealt...
2      https://insights.blackcoffer.com/is-telemedici...
3      https://insights.blackcoffer.com/is-telehealth...
4      https://insights.blackcoffer.com/how-people-di...
                             ...                        
145    https://insights.blackcoffer.com/blockchain-fo...
146    https://insights.blackcoffer.com/the-future-of...
147    https://insights.blackcoffer.com/big-data-anal...
148    https://insights.blackcoffer.com/business-anal...
149    https://insights.blackcoffer.com/challenges-an...
Name: URL, Length: 150, dtype: object

In [6]:
# page=requests.get(URLs[0] , headers={"User-Agent": "XY"})
# page

In [7]:
# Loading positive words
with open('../MasterDictionary/positive-words.txt','r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')

# Loading negative words
with open('../MasterDictionary/negative-words.txt','r' ,  encoding="ISO-8859-1") as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

#Loading stop words dictionary for removing stop words
# GENERIC
with open('../StopWords/StopWords_Generic.txt','r' ,  encoding="ISO-8859-1") as genericfile:
    genericword=genericfile.read().lower()
GenericWordList=genericword.split('\n')

# GENERIC LONG
with open('../StopWords/StopWords_GenericLong.txt','r' ,  encoding="ISO-8859-1") as genericlongfile:
    genericlongword=genericlongfile.read().lower()
GenericLongWordList=genericlongword.split('\n')

# NAMES
with open('../StopWords/StopWords_Names.txt','r' ,  encoding="ISO-8859-1") as namefile:
    names=namefile.read().lower()
nameList=names.split('\n')

# GEOGRAPHIC
with open('../StopWords/StopWords_Geographic.txt','r' ,  encoding="ISO-8859-1") as geofile:
    geographics=geofile.read().lower()
geographicList=geographics.split('\n')

# GENERIC DATES
with open('../StopWords/StopWords_DatesandNumbers.txt','r' ,  encoding="ISO-8859-1") as datefile:
    date=datefile.read().lower()
dateList=date.split('\n')

# CURRENCIES
with open('../StopWords/StopWords_Currencies.txt','r' ,  encoding="ISO-8859-1") as currenciesfile:
    currencies=currenciesfile.read().lower()
currenciesList=currencies.split('\n')

# AUDITOR
with open('../StopWords/StopWords_Auditor.txt','r' ,  encoding="ISO-8859-1") as auditfile:
    audit=auditfile.read().lower()
auditList=audit.split('\n')
# with open(stopWordsFile ,'r') as stop_words:
#     stopWords = stop_words.read().lower()
# stopWordList = stopWords.split('\n')
# stopWordList[-1:] = []

In [8]:
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in GenericWordList, tokens))
    filtered_words = list(filter(lambda token: token not in GenericLongWordList, tokens))
    filtered_words = list(filter(lambda token: token not in nameList, tokens))
    filtered_words = list(filter(lambda token: token not in geographicList, tokens))
    filtered_words = list(filter(lambda token: token not in dateList, tokens))
    filtered_words = list(filter(lambda token: token not in currenciesList, tokens))
    filtered_words = list(filter(lambda token: token not in auditList, tokens))
    return filtered_words

def positive_score(text):
    posword=0
    tokenphrase = tokenizer(text)
    for word in tokenphrase:
        if word in positiveWordList:
            posword+=1
    retpos = posword
    return retpos 

def negative_score(text):
    negword=0
    tokenphrase = tokenizer(text)
    for word in tokenphrase:
        if word in negativeWordList: 
            negword +=1
    retneg = negword 
    return retneg

def polarity_score(positive_score , negative_score):
    return (positive_score - negative_score)/((positive_score + negative_score) + 0.000001)

def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

def AverageSentenceLength (text):
    Wordcount = len(tokenizer(text))
    SentenceCount = len(sent_tokenize(text))
    if SentenceCount > 0: 
        Average_Sentence_Lenght = Wordcount / SentenceCount
    avg = Average_Sentence_Lenght
    return round(avg)

def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex

def subjectivity(pos,neg,total):
    return (pos + neg)/ ((total) + 0.000001)

In [9]:
 def avg_word_length(content):
    return len(content.replace(' ',''))/len(content.split())

In [10]:
def ProperNounExtractor(text):
    count = 0
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged = nltk.pos_tag(words)
        for (word, tag) in tagged:
            if tag == 'PRP': # If the word is a proper noun
                count = count + 1 
    return(count)


def syllable_count(content):
    word=content.replace(' ','')
    syllable_count=0
    for w in word:
        if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
    syllable_per_word = syllable_count/len(content.split())
    return syllable_per_word

In [11]:
def avg_words_per_sent(content):
    AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
    AVG_NUMBER_OF_WORDS_PER_SENTENCE=sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE)
    return AVG_NUMBER_OF_WORDS_PER_SENTENCE

In [12]:
txt_data = [(open(str(i)+'.txt', "r", encoding="ISO-8859-1")).read().lower() for i in range(2,151)]
  
# Create the pandas DataFrame with column name is provided explicitly
df = pd.DataFrame(txt_data, columns=['text'])
  
# print dataframe.
df

,text
0,\nthe last 2 years are the most critical time ...
1,"\ntelemedicine, which allows patients and doct..."
2,\ntelemedicine refers to a specific set of cli...
3,\nthe future of telehealth stays positive\nto ...
4,\nimagine a future where you could see any rem...
...,...
144,\nreconciling with the financial realities of ...
145,\nwhat is an investment?\nan investment is a r...
146,\nquality and affordable healthcare is a visio...
147,\nanalytics is a statistical scientific proces...


In [13]:
df["tokens"] = df["text"].apply(tokenizer)

In [15]:
df["AverageSentenceLength"] = df["text"].apply(AverageSentenceLength)
df["percentage_complex_word"] = df["text"].apply(percentage_complex_word)
df["fog_index"] = np.vectorize(fog_index)(df['AverageSentenceLength'],df['percentage_complex_word'])
df["complex_word_count"] = df["text"].apply(complex_word_count)
# df["total word count"] = df["text"].apply(total_word_count)
df["total_word_count"] = df["text"].apply(total_word_count)
df["positive_score"] = df["text"].apply(positive_score)
df["negative_score"] = df["text"].apply(negative_score)
df["polarity_score"] = np.vectorize(polarity_score)(df['positive_score'],df['negative_score'])
df["subjectivity"] = np.vectorize(subjectivity)(df['positive_score'],df['negative_score'], df['total_word_count'])

In [16]:
df['avg_word_length']=df['text'].apply(avg_word_length)

In [17]:
df['personal_pronouns'] = df['text'].apply(ProperNounExtractor)

In [18]:
df['syllable_per_word'] = df['text'].apply(syllable_count)

In [19]:
df['avg_words_per_sent'] = df['text'].apply(avg_words_per_sent)

In [20]:
df.head(5)

,text,tokens,AverageSentenceLength,percentage_complex_word,fog_index,complex_word_count,positive_score,negative_score,polarity_score,total_word_count,subjectivity,avg_word_length,personal_pronouns,syllable_per_word,avg_words_per_sent
0,\nthe last 2 years are the most critical time ...,"[the, last, 2, years, are, the, most, critical...",20,0.243381,8.097352,239,42,16,0.448276,982,0.059063,5.137500,18,2.015625,17.232143
1,"\ntelemedicine, which allows patients and doct...","[telemedicine, which, allows, patients, and, d...",19,0.300836,7.720334,540,87,28,0.513043,1795,0.064067,5.805461,26,2.308874,18.903226
2,\ntelemedicine refers to a specific set of cli...,"[telemedicine, refers, to, a, specific, set, o...",22,0.309836,8.923934,567,65,24,0.460674,1830,0.048634,5.937535,20,2.311199,17.250000
3,\nthe future of telehealth stays positive\nto ...,"[the, future, of, telehealth, stays, positive,...",26,0.280172,10.512069,520,88,34,0.442623,1856,0.065733,5.718510,17,2.214129,22.292683
4,\nimagine a future where you could see any rem...,"[imagine, a, future, where, you, could, see, a...",21,0.312500,8.525000,575,75,39,0.315789,1840,0.061957,5.861513,23,2.360957,18.608247


In [21]:
output_df = input_df[['URL_ID', 'URL']]

In [22]:
extracted_columns = df[['positive_score','negative_score','polarity_score','subjectivity','AverageSentenceLength','percentage_complex_word','fog_index','avg_words_per_sent','complex_word_count','total_word_count','syllable_per_word','personal_pronouns','avg_word_length']]

In [24]:
output_df=output_df.join(extracted_columns)

In [25]:
output_df.head(5)

,URL_ID,URL,positive_score,negative_score,polarity_score,subjectivity,AverageSentenceLength,percentage_complex_word,fog_index,avg_words_per_sent,complex_word_count,total_word_count,syllable_per_word,personal_pronouns,avg_word_length
0,1,https://insights.blackcoffer.com/is-telehealth...,42.0,16.0,0.448276,0.059063,20.0,0.243381,8.097352,17.232143,239.0,982.0,2.015625,18.0,5.137500
1,2,https://insights.blackcoffer.com/how-telehealt...,87.0,28.0,0.513043,0.064067,19.0,0.300836,7.720334,18.903226,540.0,1795.0,2.308874,26.0,5.805461
2,3,https://insights.blackcoffer.com/is-telemedici...,65.0,24.0,0.460674,0.048634,22.0,0.309836,8.923934,17.250000,567.0,1830.0,2.311199,20.0,5.937535
3,4,https://insights.blackcoffer.com/is-telehealth...,88.0,34.0,0.442623,0.065733,26.0,0.280172,10.512069,22.292683,520.0,1856.0,2.214129,17.0,5.718510
4,5,https://insights.blackcoffer.com/how-people-di...,75.0,39.0,0.315789,0.061957,21.0,0.312500,8.525000,18.608247,575.0,1840.0,2.360957,23.0,5.861513


In [27]:
output_df.to_excel('Output Data.xlsx', encoding='utf-8',index=False)